In [13]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapefile as shp
import matplotlib.pyplot as plt

file = pd.ExcelFile('Wijkdata_Jeugdhulp_in_de_wijk.ods')

sheet_names = file.sheet_names

wijk = pd.read_excel(file, sheet_name=sheet_names[-3])  # wijk
gebr = pd.read_excel(file, sheet_name=sheet_names[-2])  # gebruik
det = pd.read_excel(file, sheet_name=sheet_names[-1])  # determinanten

file.close()

df = det.copy()
# als je wijk en gemeentecode hebt heb je wijkcode en gemeentenaam niet nodig
df = df.merge(wijk[['wijk', 'gemeentecode']], on='wijk', how='left')
# per_jhzv : aandeel jeugdigen met jeugdhulp zonder verblijf, waargenomen -> target variable
df = df.merge(gebr[['wijk', 'perc_jhzv']], on='wijk', how='left')

for col in df.columns:
    df[col] = df[col].apply(lambda x: x if x != '.' else np.nan)

# TODO is nu missig values vullen met gemiddelde hoe willen we dat doen?
df.fillna(df.mean(), inplace=True)

# hun schatting root mean squared error (residu = schatting - waargenomen)
for col in gebr.columns:
    gebr[col] = gebr[col].apply(lambda x: x if x != '.' else np.nan)

# drop missende schattingen en bereken RMSE
# original_rmse = str((gebr.dropna(subset='residu')['residu']**2).mean()**0.5)

# buurt_geo = gpd.read_file('./WijkBuurtkaart_2022_v1/buurt_2022_v1.shp')
wijk_geo = gpd.read_file('./WijkBuurtkaart_2022_v1/wijk_2022_v1.shp')
# gemeente_geo = gpd.read_file('./WijkBuurtkaart_2022_v1/gemeente_2022_v1.shp')

wijk_geo['WK_CODE'] = wijk_geo['WK_CODE'].apply(lambda x: x[2:]).astype('float')
wijk_geo['GM_CODE'] = wijk_geo['GM_CODE'].apply(lambda x: x[2:]).astype('float')
wijk_final = df.merge(wijk_geo, left_on=['wijk', 'gemeentecode'], right_on=['WK_CODE', 'GM_CODE'], how='left')
for col in wijk_final.columns:
    wijk_final[col] = wijk_final[col].apply(lambda x: x if x != int(-99999999) else np.nan)

df = wijk_final.fillna(wijk_final.mean())
# df = df.drop(['JRSTATCODE', 'JAAR', 'Shape_Leng', 'Shape_Area', 'geometry', 'wijk', 'WK_CODE', 'WK_NAAM', 'GM_CODE', 'GM_NAAM', 'gemeentecode', 'H2O'], axis=1)
df

In [ ]:

wijk_geo.crs
gdf = gpd.GeoDataFrame(df, crs="EPSG:28992", geometry=df.geometry)
gdf.columns

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize = (12,12))
gdf.plot(ax=ax, column="AANT_INW",legend=True, legend_kwds={"label": "Aantal mensen per gemeente", "orientation": "horizontal"}, edgecolor="black")
print(gdf[gdf.gemeentecode == 363].WK_NAAM)

In [ ]:
gdf.crs

In [ ]:
import plotly.express as px
import pyproj

gdf.to_crs(pyproj.CRS.from_epsg(28992), inplace=True)
fig = px.choropleth(gdf,
                   geojson=gdf.geometry,
                   locations=gdf.index,
                   color="woonwaarde",
                   projection="mercator")

fig.show()